# Introduction

When investigating astronomical objects, like active galactic nuclei (AGN), astronomers compare data about those objects from different telescopes at different wavelengths.

This requires positional cross-matching to find the closest counterpart within a given radius on the sky.

In this activity you'll cross-match two catalogues: one from a radio survey, the AT20G Bright Source Sample (BSS) catalogue and one from an optical survey, the SuperCOSMOS all-sky galaxy catalogue.

The BSS catalogue lists the brightest sources from the AT20G radio survey while the SuperCOSMOS catalogue lists galaxies observed by visible light surveys. If we can find an optical match for our radio source, we are one step closer to working out what kind of object it is, e.g. a galaxy in the local Universe or a distant quasar.

We've chosen one small catalogue (BSS has only 320 objects) and one large one (SuperCOSMOS has about 240 million) to demonstrate the issues you can encounter when implementing cross-matching algorithms.

## Equatorial coordinates

The positions of stars, galaxies and other astronomical objects are usually recorded in either equatorial or Galactic coordinates.

Equatorial coordinates are fixed relative to the celestial sphere, so the positions are independent of when or where the observations took place. They are defined relative to the celestial equator (which is in the same plane as the Earth's equator) and the ecliptic (the path the sun traces throughout the year).

A point on the celestial sphere is given by two coordinates:

Right ascension: the angle from the vernal equinox to the point, going east along the celestial equator;
Declination: the angle from the celestial equator to the point, going north (negative values indicate going south).
The vernal equinox is the intersection of the celestial equator and the ecliptic where the ecliptic rises above the celestial equator going further east.

## Coordinates: Right Ascension

Right ascension is often given in hours-minutes-seconds (HMS) notation, because it was convenient to calculate when a star would appear over the horizon. A full circle in HMS notation is 24 hours, which means 1 hour in HMS notation is equal to 15 degrees.

Each hour is split into 60 minutes and each minute into 60 seconds.

You can convert 23 hours, 12 minutes and 6 seconds (written as 23:12:06 or 23h12m06s) to degrees like this:

In [1]:
print(15*(23 + 12/60 + 6/(60*60)))

348.025


## Coordinates: Declination

Declination, on the other hand, is traditionally recorded in degrees-minutes-seconds (DMS) notation. A full circle is 360 degrees, each degree has 60 arcminutes and each arcminute has 60 arcseconds.

For example: 73 degrees, 21 arcminutes and 14.4 arcseconds (written 73:21:14.4 or 73° 21' 14.4" or 73d21m14.4s) can be converted to decimal degrees like this:

In [2]:
print(73 + 21/60 + 14.4/(60*60))

73.354


With negative angles like -5° 31' 12" the negation applies to the whole angle, including arcminutes and arcseconds:

In [3]:
print(-1*(5 + 31/60 + 12/(60*60)))

-5.52


## Convert to decimal degrees

Write two functions, one that converts right ascension from HMS to decimal degrees, called hms2dec, and another that converts declination from DMS to decimal degrees, called dms2dec.

Right ascension is always an angle from 0 to 24 hours and declination is always an angle from -90° to +90°.

In [4]:
def hms2dec(h,m,s):
  return 15*(h+m/60 + s/(60*60))

def dms2dec(d,m,s):
  if d > 0:
     return d + m/60 + s/(60*60)
  else:
     return -1*(abs(d) + m/60 + s/(60*60))


In [6]:
# The first example from the question
print(hms2dec(23, 12, 6))

348.025


In [8]:
# The second example from the question
print(dms2dec(22, 57, 18))

22.955


In [9]:
print(dms2dec(-66, 5, 5.1))

-66.08475


## Angular distance

To crossmatch two catalogues we need to compare the angular distance between objects on the celestial sphere.

People loosely call this a "distance", but technically its an angular distance: the projected angle between objects as seen from Earth.

If we have an object on the celestial sphere with right ascension and declination , then the angular distance to another object with coordinates  is:

Angular distances have the same units as angles (degrees). There are other equations for calculating the angular distance but this one, called the haversine formula, is good at avoiding floating point errors when the two points are close together.

## Calculating angular distance
We'll go through an example of how to implement the formula you saw on the previous slide using NumPy's trigonometric functions.

We can calculate b with NumPy's sin, cos and abs functions:

In [11]:
import numpy as np

b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2

NameError: name 'd1' is not defined

Here, r1 and d1 are the coordinates of the first point  and r2 and d2 similarly correspond to  and .

 can be calculated in a similar way using just sin and abs. Once we have both  and  we can use numpy.arcsin to calculate :

In [12]:
d = 2*np.arcsin(np.sqrt(a + b))

NameError: name 'a' is not defined

## Heavenly angles

Write a function called angular_dist that calculates the angular distance between any two points on the celestial sphere given their right ascension and declination.

Your function should take arguments in decimal degrees and return the distance in decimal degrees too.

In [14]:
import numpy as np

def angular_dist(r1_deg,d1_deg,r2_deg,d2_deg):
  r1 = np.radians(r1_deg)
  r2 = np.radians(r2_deg)
  d1 = np.radians(d1_deg)
  d2 = np.radians(d2_deg)
  
  delta_1 = d1 - d2
  if delta_1<0:
    delta_1 = - delta_1
  delta_1 = (delta_1)/2
  a = (np.sin(delta_1))*(np.sin(delta_1))
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  d = 2*np.arcsin(np.sqrt(a + b)) 
  return np.degrees(d)


In [15]:
# Run your function with the first example in the question.
print(angular_dist(21.07, 0.1, 21.15, 8.2))

8.10039231815


In [17]:
# Run your function with the second example in the question
print(angular_dist(10.3, -3, 24.3, -29))

29.2084981805


## Loading catalogues

Before we can crossmatch our two catalogues we first have to import the raw data. Every astronomy catalogue tends to have its own unique format so we'll need to look at how to do this with each one individually.

We'll look at the AT20G bright source sample survey first. The raw data we'll be using is the file table2.dat from this page in the VizieR archives, but we'll use the filename bss.dat from now on.

Every catalogue in VizieR has a detailed README file that gives you the exact format of each table in the catalogue.

## AT20G BSS catalogue

The full catalogue of bright radio sources contains 320 objects

The catalogue is organised in fixed-width columns, with the format of the columns being:

1: Object catalogue ID number (sometimes with an asterisk)
2-4: Right ascension in HMS notation
5-7: Declination in DMS notation
8-: Other information, including spectral intensities
We only need coordinates for crossmatching. We can load specific columns with the usecols argument in NumPy's loadtxt function:

In [18]:
import numpy as np
cat = np.loadtxt('bss.dat', usecols=range(1, 7))
print(cat[0])

[  0.     4.    35.65 -47.    36.    19.1 ]


## SuperCOSMOS all-sky catalogue
The SuperCOSMOS all-sky catalogue is a catalogue of galaxies generated from several visible light surveys.

The original data is available on this page in a package called SCOS_XSC_mCl1_B21.5_R20_noStepWedges.csv.gz. Because this catalogue is so large, we've cut it down for these activities. The cut down version of the file will be named super.csv.

The catalogue uses a comma-separated value (CSV) format. Aside from the first row, which contains column labels, the format is:

1: Right ascension in decimal degrees

2: Declination in decimal degrees

3: Other data, including magnitude and apparent shape

So now when loading this file in, we have to tell np.loadtxt to skip the first row and treat the commas as delimiters:

In [19]:
import numpy as np
cat = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=[0, 1])
print(cat)

[[  1.0583407 -52.9162402]
 [  2.6084425 -41.5005753]
 [  2.7302499 -27.706955 ]]


## Playing with space cats

write import_bss and import_super functions that import the AT20G BSS and SuperCOSMOS catalogues from the files bss.dat and super.csv as described in the previous slides.

Each function should return a list of tuples containing the object's ID (an integer) and the coordinates in degrees. The object ID should be the row of the object in the catalogue, starting at 1.



In [22]:
import numpy as np

def hms2dec(h,m,s):
  return 15*(h+(m/60)+(s/3600))

def dms2dec(d,m,s):
  if d>0:
    return (d+(m/60)+(s/3600))
  else:
    return -((-d)+(m/60)+(s/3600))

def import_bss():
  cat = np.loadtxt('bss.dat',usecols=range(1,7))
  results= []
  i = 1
  for item in cat:
    h1 = hms2dec(item[0],item[1],item[2])
    r1 = dms2dec(item[3],item[4],item[5])
    results.append((i,h1,r1))
    i = i+1
  return results

def import_super():
  cat = np.loadtxt('super.csv',delimiter = ',',skiprows = 1, usecols = [0,1])
  results = []
  i = 1
  for item in cat:
    results.append((i,item[0],item[1]))
    i = i+1
  return results


In [24]:
bss_cat = import_bss()
super_cat = import_super()

bss_cat
super_cat

[(1, 1.0583407, -52.916240199999997),
 (2, 2.6084425000000002, -41.500575300000001),
 (3, 2.7302499, -27.706955000000001)]

## Look around you
Write a find_closest function that takes a catalogue and the position of a target source (a right ascension and declination) and finds the closest match for the target source in the catalogue.

Your function should return the ID of the closest object and the distance to that object.

The right ascension and declination are in degrees. The catalogue list has been loaded by import_bss from the previous question. The full 320 object BSS catalogue is contained in bss.dat for you to test your code on.

In [28]:
import numpy as np
def angular_dist(r1_deg,d1_deg,r2_deg,d2_deg):
  r1 = np.radians(r1_deg)
  r2 = np.radians(r2_deg)
  d1 = np.radians(d1_deg)
  d2 = np.radians(d2_deg)
  
  delta_1 = d1 - d2
  if delta_1<0:
    delta_1 = - delta_1
  delta_1 = (delta_1)/2
  a = (np.sin(delta_1))*(np.sin(delta_1))
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  d = 2*np.arcsin(np.sqrt(a + b)) 
  return np.degrees(d)

def hms2dec(h,m,s):
  return 15*(h+(m/60)+(s/3600))

def dms2dec(d,m,s):
  if d>0:
    return (d+(m/60)+(s/3600))
  else:
    return -((-d)+(m/60)+(s/3600))

def import_bss():
  cat = np.loadtxt('bss.dat',usecols=range(1,7))
  results = []
  i = 1
  for item in cat:
    h1 = hms2dec(item[0],item[1],item[2])
    r1 = dms2dec(item[3],item[4],item[5])
    results.append((i,h1,r1))
    i = i+1
  return results

def find_closest(cat,r,d):
  item_select = cat[1]
  lowest_dist = angular_dist(r,d,item_select[1],item_select[2])
  
  for item in cat:
    item_dist = angular_dist(r,d,item[1],item[2])
    if item_dist<lowest_dist:
      lowest_dist = item_dist
      item_select = item
  return (item_select[0],lowest_dist)


In [29]:
cat = import_bss()
  
  # First example from the question
print(find_closest(cat, 175.3, -32.5))

  # Second example in the question
print(find_closest(cat, 32.2, 40.7))


(1, 99.722586584426395)
(3, 72.282148208126713)


## Matchup
Write a crossmatch function that crossmatches two catalogues within a maximum distance. It should return a list of matches and non-matches for the first catalogue against the second.

The list of matches contains tuples of the first and second catalogue object IDs and their distance. The list of non-matches contains the unmatched object IDs from the first catalogue only. Both lists should be ordered by the first catalogue's IDs.

The BSS and SuperCOSMOS catalogues will be given as input arguments, each in the format you’ve seen previously. The maximum distance is given in decimal degrees.

In [30]:
import numpy as np

def import_super():
  cat = np.loadtxt('super.csv',delimiter = ',',skiprows = 1, usecols = [0,1])
  list_ele = []
  i = 1
  for item in cat:
    list_ele.append((i,item[0],item[1]))
    i = i+1
  return list_ele

def angular_dist(r1_deg,d1_deg,r2_deg,d2_deg):
  r1 = np.radians(r1_deg)
  r2 = np.radians(r2_deg)
  d1 = np.radians(d1_deg)
  d2 = np.radians(d2_deg)
  
  delta_1 = d1 - d2
  if delta_1<0:
    delta_1 = - delta_1
  delta_1 = (delta_1)/2
  a = (np.sin(delta_1))*(np.sin(delta_1))
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  d = 2*np.arcsin(np.sqrt(a + b)) 
  return np.degrees(d)

def hms2dec(h,m,s):
  return 15*(h+(m/60)+(s/3600))

def dms2dec(d,m,s):
  if d>0:
    return (d+(m/60)+(s/3600))
  else:
    return -((-d)+(m/60)+(s/3600))

def import_bss():
  cat = np.loadtxt('bss.dat',usecols=range(1,7))
  results = []
  i = 1
  for item in cat:
    h1 = hms2dec(item[0],item[1],item[2])
    r1 = dms2dec(item[3],item[4],item[5])
    results.append((i,h1,r1))
    i = i+1
  return results

def find_closest(cat,r,d):
  item_select = cat[1]
  lowest_dist = angular_dist(r,d,item_select[1],item_select[2])
  
  for item in cat:
    item_dist = angular_dist(r,d,item[1],item[2])
    if item_dist<lowest_dist:
      lowest_dist = item_dist
      item_select = item
  return (item_select[0],lowest_dist)

def crossmatch(bss_cat,super_cat,max_dist):
  matches = []
  no_matches = []
  for bss_item in bss_cat:
    closest = find_closest(super_cat,bss_item[1],bss_item[2])
    if closest[1] < max_dist:
      matches.append((bss_item[0],closest[0],closest[1]))
    else:
      no_matches.append(bss_item[0])
  
  return matches,no_matches

In [31]:
bss_cat = import_bss()
super_cat = import_super()

  # First example in the question
max_dist = 5
matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
print(matches[:3])
print(no_matches[:3])
print(len(no_matches))

  # Second example in the question
max_dist = 1
matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
print(matches[:3])
print(no_matches[:3])
print(len(no_matches))

[(2, 3, 2.7573607427578017), (3, 3, 1.4979268488836439)]
[1]
1
[]
[1, 2, 3]
3
